In [1]:
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [2]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [3]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model="gpt-5-nano",
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [4]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="제 이메일을 보고 답장 이메일을 보내주세요")],
        "email": "안녕하세요, 저 내일 미팅 늦을 것 같아요. 일정 조정 가능할까요? 유광명 드림."
    },
    config=config
)

In [5]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': '제목: '
                                                                          'Re: '
                                                                          '내일 '
                                                                          '미팅 '
                                                                          '관련\n'
                                                                          '\n'
                                                                          '안녕하세요, '
                                                                          '유광명 '
                                                                          '님.\n'
                                                                          '\n'
                                                                          '내일 '
                                                                          '미팅에 '
                                

- 인터럽트가 걸려있고, 마지막 AI 메시지는 content가 없는 도구 호출 메시지다.

In [6]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': '제목: Re: 내일 미팅 관련\n\n안녕하세요, 유광명 님.\n\n내일 미팅에 늦으신다니 이해합니다. 일정 조정이 가능하도록 아래 중 편하신 옵션을 알려주시면 바로 반영하겠습니다.\n\n- 현재 예정된 시간에서 10~15분 정도 지연 가능하신가요?\n- 더 지연이 필요하신 경우, 가능한 대체 시간대가 있으시면 알려 주세요. 예: 11:00, 11:30 등\n\n도착 예정 시간 ETA를 간단히 알려주시면 팀에 공유하겠습니다.\n\n감사합니다.'}, 'description': "Tool execution requires approval\n\nTool: send_email\nArgs: {'body': '제목: Re: 내일 미팅 관련\\n\\n안녕하세요, 유광명 님.\\n\\n내일 미팅에 늦으신다니 이해합니다. 일정 조정이 가능하도록 아래 중 편하신 옵션을 알려주시면 바로 반영하겠습니다.\\n\\n- 현재 예정된 시간에서 10~15분 정도 지연 가능하신가요?\\n- 더 지연이 필요하신 경우, 가능한 대체 시간대가 있으시면 알려 주세요. 예: 11:00, 11:30 등\\n\\n도착 예정 시간 ETA를 간단히 알려주시면 팀에 공유하겠습니다.\\n\\n감사합니다.'}"}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='4504f769072ee90fa20edc902c001bc6')]


In [7]:
# 인터럽트 걸린 내용: 우리가 보내려고 하는 이메일 내용.
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

제목: Re: 내일 미팅 관련

안녕하세요, 유광명 님.

내일 미팅에 늦으신다니 이해합니다. 일정 조정이 가능하도록 아래 중 편하신 옵션을 알려주시면 바로 반영하겠습니다.

- 현재 예정된 시간에서 10~15분 정도 지연 가능하신가요?
- 더 지연이 필요하신 경우, 가능한 대체 시간대가 있으시면 알려 주세요. 예: 11:00, 11:30 등

도착 예정 시간 ETA를 간단히 알려주시면 팀에 공유하겠습니다.

감사합니다.


## Approve

In [ ]:
from langgraph.types import Command


response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': '안녕하세요, 저 내일 미팅 늦을 것 같아요. 일정 조정 가능할까요? 유광명 드림.',
 'messages': [HumanMessage(content='제 이메일을 보고 답장 이메일을 보내주세요', additional_kwargs={}, response_metadata={}, id='d4f2b2ca-1c20-494f-8555-d7f6d62e72ae'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 915, 'prompt_tokens': 158, 'total_tokens': 1073, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 896, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D794WPSEvgmgz0NyKGzSRAW7yMBr5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3fb7-dafe-7bf0-9353-c0460f8793c4-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_RWoe1YKgY5EagecVEIvCNSOY', 'type': 'tool_call'}], invalid_tool_calls=[], 

## Reject

In [9]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # 거절 사유 입력
                    "message": "아니요. 서명을 추가하세요 - 홍길동 드림."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': '안녕하세요, 저 내일 미팅 늦을 것 같아요. 일정 조정 가능할까요? 유광명 드림.',
 'messages': [HumanMessage(content='제 이메일을 보고 답장 이메일을 보내주세요', additional_kwargs={}, response_metadata={}, id='d4f2b2ca-1c20-494f-8555-d7f6d62e72ae'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 915, 'prompt_tokens': 158, 'total_tokens': 1073, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 896, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D794WPSEvgmgz0NyKGzSRAW7yMBr5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c3fb7-dafe-7bf0-9353-c0460f8793c4-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'call_RWoe1YKgY5EagecVEIvCNSOY', 'type': 'tool_call'}], invalid_tool_calls=[], 

In [10]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'

## Edit

In [ ]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)